In [1]:
# CMSE Project
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import random as random
import statsmodels.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects
import statsmodels.formula.api as smf

In [2]:
# Reading in Data
# Keys to extract formatted State and County names
key = pd.read_excel('fips_codes_website.xls')   # Counties and major cities
key00 = pd.read_csv('FIPS_countycodes.txt',delimiter=',')  # Counties only
key00['GU Name'] = key00['GU Name'].str[:-7]  # Slice 'county' from names
# Housing Data
zillow2019 = pd.read_excel("County_Zhvi_Summary_AllHomes2019.xls")
zillow2019['RegionName'] = zillow2019['RegionName'].str[:-7]  # Slice 'county' from names
# Crime
crime2010 = pd.read_csv('33523-0001-crimedata2010.txt',delimiter='\t')
crime2012 = pd.read_csv('35019-0001-crimedata2012.txt',delimiter='\t')
crime2014 = pd.read_csv('36399-0001-crimedata2014.txt',delimiter='\t')
crime2016 = pd.read_csv('37059-0001-crimedata2016.txt',delimiter='\t')
# Enumployment & Wages
#test_wages_18 = pd.read_excel('allhlcn183.xlsx',sheet_name='US_St_Cn_MSA')
wage_unemp2018 = pd.read_csv('allhlcn183wage_unemp2018.txt',delimiter='\t',dtype={'St':str})
wage_unemp2016 = pd.read_csv('allhlcn163wage_unemp2016.txt',delimiter='\t',dtype={'St':str})
wage_unemp2014 = pd.read_csv('allhlcn143wage_unemp2014.txt',delimiter='\t',dtype={'St':str})
wage_unemp2012 = pd.read_csv('allhlcn123wage_unemp2012.txt',delimiter='\t',dtype={'St':str})
wage_unemp2010 = pd.read_csv('allhlcn103wage_unemp2010.txt',delimiter='\t',dtype={'St':str})

print("2014 crime:",crime2014.shape,"// key:",key.shape)
print("2019 Housing:",zillow2019.shape)
print('2018 Wages:',wage_unemp2018.shape)
x = ((crime2014.shape[0])*(crime2014.shape[1]))*4 + ((wage_unemp2018.shape[0])*(wage_unemp2018.shape[1]))*5 + (key00.shape[0])*key00.shape[1]
print(x,'unique datapoints in all datasets')

2014 crime: (3177, 56) // key: (41787, 7)
2019 Housing: (1921, 17)
2018 Wages: (62724, 21)
7313843 unique datapoints in all datasets


In [ ]:
#[i[i.find(' ')+1:] for i in test_wages_18['Industry']]

In [ ]:
def crime_cleaner(data):
    # Merge key with crime db to gain state / city names
    washing = data.merge(key00,left_on=['FIPS_CTY','FIPS_ST'], right_on=['County FIPS Code','State FIPS Code'])
    # Rearrange columns
    # for 'key'
    #washing = washing[['STUDYNO','EDITION','PART','IDNO','FIPS_ST','State FIPS Code','State Abbreviation','FIPS_CTY','County FIPS Code','GU Name','Entity Description','CPOPARST','AG_ARRST','JURFLAG','COVIND', 'GRNDTOT','P1TOT','P1VLNT','P1PRPTY','MURDER','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY','FIPS Entity Code','ANSI Code']]
    # for 'key00'
    washing = washing[['STUDYNO','EDITION','PART','IDNO','FIPS_ST','State FIPS Code','State Abbreviation','FIPS_CTY','County FIPS Code','GU Name','CPOPARST','AG_ARRST','JURFLAG','COVIND', 'GRNDTOT','P1TOT','P1VLNT','P1PRPTY','MURDER','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY']]
    
    # Select only wanted columns
    clean = washing[['State Abbreviation','GU Name','MURDER','P1TOT','P1VLNT','P1PRPTY','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY']]
    return clean
def wage_cleaner(data):
    # Drop Unwanted columns
    base = data.drop(['Area\nCode','Own','NAICS','Qtr','Status Code'],axis=1)
    # Select County level data
    washing = base.loc[(base['Area Type'] == 'County')]
    washing['Cnty'] = washing['Cnty'].astype('int64') # Convert to int for merging // Throws error
    washing['St'] = washing['St'].astype('int64')    # Convert to into for merging // Throws error
    # Merge with key
    washing = washing.merge(key00,left_on=['St','Cnty'], right_on=['State FIPS Code','County FIPS Code'])
    # Rearrange columns and drop unwanted columns
    county_wages = washing[['State Abbreviation','GU Name', 'Year', 'Area Type', 'St Name', 'Area', 'Ownership','Industry', 'Establishment Count', 'July Employment','August Employment', 'September Employment', 'Total Quarterly Wages','Average Weekly Wage', 'Employment Location Quotient Relative to U.S.','Total Wage Location Quotient Relative to U.S.']]
    county_wages['Establishment Count'] = county_wages['Establishment Count'].str.replace(',','')  # Get rid of comma // Throws Error
    county_wages['Establishment Count'] = pd.to_numeric(county_wages['Establishment Count'])   # Convert to interger // Throws Error
    county_wages['September Employment'] = county_wages['September Employment'].str.replace(',','')  # Get rid of comma // Throws Error
    county_wages['September Employment'] = pd.to_numeric(county_wages['September Employment'])    # Convert to interger // Throws Error  
    county_wages['Average Weekly Wage'] = county_wages['Average Weekly Wage'].str.replace(',','')  # Get rid of comma // Throws Error
    county_wages['Average Weekly Wage'] = pd.to_numeric(county_wages['Average Weekly Wage'])  # Convert to interger // Throws Error
    county_wages['Industry'] = county_wages['Industry'].str.replace('0','') # Formatting to fix year 2018 industry codes
    county_wages['Industry'] = county_wages['Industry'].str.replace('1','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('2','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('3','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('4','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('5','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('6','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('7','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('8','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('9','')
    
    state_wages = base.loc[(base['Area Type'] == 'State')]   # State Level Wages
    nation_wages = base.loc[(base['Area Type'] == 'Nation')]    # Nation Level Data
    urban_wages = base.loc[(base['Area Type'] == 'MSA')]   # Metropolitan Level Data
    return county_wages,state_wages,nation_wages,urban_wages

def search(county,state):
    # Collect crime data from all years
    crime1 = crime10.loc[(crime10['GU Name'] == county) & (crime10['State Abbreviation'] == state)]
    crime2 = crime12.loc[(crime12['GU Name'] == county) & (crime12['State Abbreviation'] == state)]
    crime3 = crime14.loc[(crime14['GU Name'] == county) & (crime14['State Abbreviation'] == state)]
    crime4 = crime16.loc[(crime16['GU Name'] == county) & (crime16['State Abbreviation'] == state)]
    crime = crime1,crime2,crime3,crime4
    # Collect wage and firm data from all years :: wage[0] pulls county level data, change to pull state, national, or urban levels
    wage1 = wage10[0].loc[(wage10[0]['GU Name'] == county) & (wage10[0]['State Abbreviation'] == state)]
    wage2 = wage12[0].loc[(wage12[0]['GU Name'] == county) & (wage12[0]['State Abbreviation'] == state)]
    wage3 = wage14[0].loc[(wage14[0]['GU Name'] == county) & (wage14[0]['State Abbreviation'] == state)]
    wage4 = wage16[0].loc[(wage16[0]['GU Name'] == county) & (wage16[0]['State Abbreviation'] == state)]
    wage5 = wage18[0].loc[(wage18[0]['GU Name'] == county) & (wage18[0]['State Abbreviation'] == state)]
    wages = wage1,wage2,wage3,wage4,wage5
    return {'Crime':crime,'Industry':wages}

# Yearly cleaned databases
crime10 = crime_cleaner(crime2010)
crime12 = crime_cleaner(crime2012)
crime14 = crime_cleaner(crime2014)
crime16 = crime_cleaner(crime2016)
cols = list(crime14.columns.values) # Grab columns
# Cleaned Wage Data
wage10 = wage_cleaner(wage_unemp2010) # Returns 3 items; county, state, national, and urban level data sets
wage12 = wage_cleaner(wage_unemp2012)
wage14 = wage_cleaner(wage_unemp2014)
wage16 = wage_cleaner(wage_unemp2016)
wage18 = wage_cleaner(wage_unemp2018)

#wage18[0]['Industry'] = [i[i.find(' ')+1:] for i in wage18[0]['Industry']] # Get to work
wage18[0]['Industry'] = wage18[0]['Industry'].str[1:] # Remove first space in 2018 industry codes

In [ ]:
# Hide half of data for analysis / statistical learning
# Generate Random Index values
dbindex = []
for i in range(int(len(crime2010['FIPS_CTY'])/2)):
               r = random.randint(0,len(crime2010['FIPS_CTY']))
               dbindex.append(r)
hidden_data = pd.DataFrame.empty
testing_data = pd.DataFrame.empty

# Generate Sample of Randomly Selected Counties
n = 5 # Numer of random samples
print("Randomly Generated Cities:\nState County")
for i in range(n):
    r = random.randint(0,len(crime12))
    print(crime12['State Abbreviation'].iloc[r]+', '+crime12['GU Name'].iloc[r])
#print(dbindex)

In [ ]:
# S E A R C H #
##############
print("IF USING 'KEY00', COUNTIES ONLY")
CountCit = input("Enter a county: ")
state = input("Enter a state abbreviation: ")

# Search indexing || Crime data 2010-2016, Wages 2010-2018,
collect = search(CountCit,state)
# Indexing of 'Collect' :: [crime / wages][data by year]

In [ ]:
# Selects 10 Random counties and stores them as a dictionary by count // Key = MI, County
samp_db = {}
for n in range(10):
    r = random.randint(0,len(crime12))
    x = crime12['State Abbreviation'].iloc[r] # Pull State Abreviation
    y = crime12['GU Name'].iloc[r]  # Pull analogous county name
    z = x+', '+y # Format name for dictionary key
    agg = search(y,x) # Return criminal and economic data
    samp_db[z] = agg  # Add data to empty dictionary

In [ ]:
print(samp_db.keys())

agg_crimes = []  # List of crime coutns for all counties in sample database
for n in samp_db:  # For each county in sample
    for i in crimes:  # For each crime in that county
        tsearch = i
        agg_crimes.append(searchcrime_cleaner(samp_db[n],crimes,tsearch))  # Pull data
     
# Combined into list-> database
#test_dic = {}
# for i in samp_db:
#     for n in industry:
#         test_dic[i+" wages"] = wage_dic[i][:-1]
#         test_dic[i+" jobs"] = emp_dic[i][:-1]
print("Crimes",crimes)
print("            CNTY01 // DRGPOSS //             AGASSLT //            DUI")
print("Aggregate Crimes: ",agg_crimes)
print("Collect: ",collect['Crime'][0])

In [ ]:
# # Multiple Regression Testing
# multireg_dic = {}    
# for i in wage_dic:  # For each industry in the wage dictionary
#     multireg_dic[i+" wages"] = wage_dic[i][:-1] # Add wage data to new dictionary, retitle for calraification
#     multireg_dic[i+" jobs"] = emp_dic[i][:-1]  # Add employment numbers per industry, retitle for calarification
# # Now we have employment and wage data in a single dictionary    

# multireg = pd.DataFrame(data=multireg_dic)  # Convert dictionary into a dataframe
# x = multireg[list(multireg.columns.values)[:3]]  # Use only the first 3 columns to regress
# y = crime_dic['DUI']  # Set y variable

# model = sm.OLS(y, x).fit()
# predictions = model.predict(x) # make the predictions by the model

In [ ]:
# Arrest Options
collect['Crime'][0].columns

In [ ]:
# Test Plotting & Collecting Specific Data
crimes = 'DRGPOSS','AGASSLT','DUI'
def searchcrime_cleaner(data,crimes,tsearch):
    container = []
    count = 0
    for i in range(len(data['Crime'])):
        knight = data['Crime'][count][tsearch].values
        container.append(knight)
        count += 1
    container = container[0][0],container[1][0],container[2][0],container[3][0]
    return container
# Pull cleaned crime data, store in dictionary
crime_dic = {}
for i in crimes:
    tsearch = i
    crime_dic[i] = searchcrime_cleaner(collect,crimes,tsearch)


name = collect['Crime'][0]['GU Name'].values, collect['Crime'][0]['State Abbreviation'].values
name = name[0]+" County, "+name[1]
years = (2010,2012,2014,2016)
#Plotting
for i in crime_dic:
    plt.plot(years,crime_dic[i],label=i)
    plt.suptitle("Plot of Crime",size=16)
    plt.title(name[0], size=12)
    plt.xlabel("Years")
    plt.ylabel("Number of Arrests")
    plt.grid()
    plt.legend()
#plt.savefig('name')
houdata = zillow2019.loc[(zillow2019['RegionName'].values == CountCit) & (zillow2019['State'].values == state)]
try:
    print("Average house price, 2019:  $",int(houdata['Zhvi']))
    print("Metro areas:",houdata['Metro'].values[0])
except:
    print('--No Housing Data Available For This County--')
print(crime_dic)

In [ ]:
# Graphics for Industry and economic Data
collect['Industry'][0].columns    # Statistic options

In [ ]:
# Function for cleaning data from search variable
def search_industrydata_cleaner(data,stat,tsearch):
    container = []
    count = 0
    for n in range(len(data['Industry'])):    # Loop for pulling data of multiple years      
        container.append(data['Industry'][n].loc[(data['Industry'][n]['Industry']==tsearch)][stat].values)
#                          dictionary//year       dictionary//year//column    comparison/DATA TO PULL
        count += 1
    container = container[0][0],container[1][0],container[2][0],container[3][0],container[4][0]
    return container # Return a list of individual values

# Function to find largest industries per county
def most_establishments(data):
    data = data.loc[(data['Industry'] != '10 Total, all industries')] # Remove Totals, 2018
    data = data.loc[(data['Industry'] != 'Total, all industries')] # Remove Totals, all other years
    data = data.nlargest(4,'Establishment Count')
    return data

industry = most_establishments(collect['Industry'][4])  # Returns 4 largest industries in 2018 ,index = 4
industrydata = industry # Data set for other metrics in largest industries
industry = list(industry['Industry']) # List of largest industries for reference purposes

In [ ]:
stat = 'Establishment Count'  
# Statistic associated with industry:: Establishment Count,Average Weekly Wage,September Employment
indus_dic = {}
for i in industry:
    tsearch = i
    rook = search_industrydata_cleaner(collect,stat,tsearch)
    indus_dic[i] = rook
# Plotting Searched Data
name = collect['Industry'][0]['GU Name'].values,collect['Industry'][0]['State Abbreviation'].values
name = name[0][0]+" County, "+name[1][0]
years = 2010,2012,2014,2016,2018
for i in industry:
    plt.plot(years,indus_dic[i],label=i)
    plt.suptitle('Number of Establishments per 4 Biggest Industries')
    plt.title(name)
    plt.legend()
plt.grid()
#plt.savefig('name')
indus_dic

In [ ]:
# Call Employment numbers
stat = 'September Employment'  
# Statistic associated with industry:: Establishment Count,Average Weekly Wage,September Employment
emp_dic = {}
for i in industry:
    tsearch = i
    rook = search_industrydata_cleaner(collect,stat,tsearch)
    emp_dic[i] = rook
# Plot
for i in industry:
    plt.plot(years,emp_dic[i],label=i)
    plt.suptitle('Number of Jobs in Largest Industries')
    plt.ylabel('Number of People Employed')
    plt.xlabel('Year')
    plt.title(name)
    plt.legend()
plt.grid()
#plt.savefig('Industries with most employed')

In [ ]:
# Call Wage Numbers
stat = 'Average Weekly Wage'  
# Statistic associated with industry:: Establishment Count,Average Weekly Wage,September Employment
wage_dic = {}
for i in industry:
    tsearch = i
    rook = search_industrydata_cleaner(collect,stat,tsearch)
    wage_dic[i] = rook
# Plot
for i in industry:
    plt.plot(years,wage_dic[i],label=i)
    plt.suptitle('Average Weekly Wages per Industry')
    plt.ylabel('Average Weekly Wages')
    plt.xlabel('Year')
    plt.title(name)
    plt.legend()
plt.grid()
print(wage_dic)

In [ ]:
indus_dic

In [ ]:
ind_vars = crime_dic['DRGPOSS']
dep_vars = emp_dic['Service-providing'][:-1] 
# Number of establishments per service providing industry
# Number of Drug Possesions in county

# Note the difference in argument order
model = sm.OLS(dep_vars, ind_vars).fit()
predictions = model.predict(ind_vars) # make the predictions by the model

# Print out the statistics
print(model.summary())

fig, ax1 = plt.subplots()
ax1.plot(years[:-1],crime_dic['DRGPOSS'],color='b',label='Number of Drug Posession Arrests')
ax1.set_ylabel('Number of Drug Posession Arrests', color='b')
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(years[:-1],emp_dic['Service-providing'][:-1],color='r',label='Number of Service Industry Establishments')
ax2.set_ylabel('Number Employed in Service Industry', color='r')
plt.title(name)
plt.grid()
print("\n Number of Jobs in the Service Industry plotted against number of drug posession arrests.")

In [ ]:
wage_dic

In [ ]:
# Multiple Regression Testing
multireg_dic = {}    
for i in wage_dic:  # For each industry in the wage dictionary
    multireg_dic[i+" wages"] = wage_dic[i][:-1] # Add wage data to new dictionary, retitle for calraification
    multireg_dic[i+" jobs"] = emp_dic[i][:-1]  # Add employment numbers per industry, retitle for calarification
# Now we have employment and wage data in a single dictionary    

multireg = pd.DataFrame(data=multireg_dic)  # Convert dictionary into a dataframe
x = multireg[list(multireg.columns.values)[:3]]  # Use only the first 3 columns to regress
y = crime_dic['DUI']  # Set y variable

model = sm.OLS(y, x).fit()
predictions = model.predict(x) # make the predictions by the model

print('Crime Dictionary (y vars): ',crime_dic)
print(model.summary())

In [ ]:
# exog_vars = test
# exog = sm.add_constant(test['Service-Wages'])
# mod = RandomEffects(y,exog)
# RE_res = mod.fit()
# print(RE_res)

In [ ]:
multireg

In [ ]:
fig, ax = plt.subplots(2)
for i in industry:
    ax[0].plot(years,wage_dic[i],label=i)
    ax[1].plot(years,emp_dic[i],label=i)
    ax[1].legend(loc='right')
print("Top graph: Wages \nBottom graph: Number of jobs")

In [ ]:
print(samp_db.keys())

agg_crimes = []  # List of crime coutns for all counties in sample database
for n in samp_db:  # For each county in sample
    for i in crimes:  # For each crime in that county
        tsearch = i
        agg_crimes.append(searchcrime_cleaner(samp_db[n],crimes,tsearch))  # Pull data
     
# 
test_dic = {}
# Create dictionary keys
for n in industry:
    test_dic["Wages "+n] = ()
    test_dic["Jobs "+n] = ()
# Pull and organize data into dictionary
for i in samp_db:
    for n in industry:
        test_dic["Wages "+n] = test_dic["Wages "+n]+wage_dic[n][:-1]
        test_dic["Jobs "+n] = test_dic["Jobs "+n]+emp_dic[n][:-1]
print("Crimes",crimes)
print("            CNTY01 // DRGPOSS //             AGASSLT //            DUI")
print("Aggregate Crimes: ",agg_crimes)
# print("Collect: ",collect['Crime'][0])

# for i in test_dic:
#     test_dic[i] = test_dic[i]+test_list
print(test_dic.keys())
print('\n\n\n',test_dic)
print(len(test_dic['Wages Service-providing']))

In [ ]:
plt.plot(test_dic["Wages Service-providing"])

In [ ]:
# wage_dic = {}
# for i in industry:
#     tsearch = i
#     rook = search_industrydata_cleaner(collect,stat,tsearch)
#     wage_dic[i] = rook

#     wage5 = wage18[0].loc[(wage18[0]['GU Name'] == county) & (wage18[0]['State Abbreviation'] == state)]
industry

In [ ]:
samp_db['NM, Lincoln']['Crime'][0]  #['P1VLNT']

In [ ]:
# Function for cleaning data sample database
def industrydata_cleaner(data,stat,tsearch):
    container = []
    for i in data:
        for n in range(len(data[i]['Industry'])-1):  # -1 because only for years of crime data
            x = data[i]['Industry'][n].loc[data[i]['Industry'][n]['Industry']== tsearch]
            x = x[stat].values
            #print(type(x))
            container.append(x)
    return container # Return a list of individual values
def sample_crime_cleaner(data,crime):
    container = []
    for i in data:
        for n in range(len(data[i]['Crime'])):
            y = data[i]['Crime'][n][crime]
            y = y.values
            #print(type(y))
            container.append(y[0])
    return container

test_dic = {}
stat = 'Establishment Count'
for n in industry:
    test_dic[n] = industrydata_cleaner(samp_db,stat,n)
    
for i in crimes:
    test_dic[i] = sample_crime_cleaner(samp_db,i)
test = pd.DataFrame(data=test_dic)
print("Sample Data: ",samp_db.keys(),'\n')
#print("Test Dictonary: ",test_dic.keys())

plt.plot(test['Service-providing'])
print('Average Weekly Wages')
test
# print("Crime db type: ",type(samp_db['NM, Lincoln']['Crime'][0]))
# print("Industry db type: ",type(samp_db['NM, Lincoln']['Industry'][0]))

In [ ]:
test.dtypes

In [ ]:
for j in test_dic['Education and health services']:
    try:
        print(j[0])
    except:
        print('')
#     print(j[0])

In [ ]:
serv_col = [i[0] for i in test_dic["Service-providing"]]
serv_colpd = pd.Series(serv_col)
test['Service-providing'] = serv_colpd

# Education and health services
edu_list = np.array([])
for i in test_dic["Education and health services"]:
    try:
        edu_list = np.append(edu_list,i[0])
    except:
        edu_list = np.append(edu_list,0)
edu_colpd = pd.Series(edu_list)
test['Education and health services'] = edu_colpd

# Professional and business services
prof_list = np.array([])
for i in test_dic["Professional and business services"]:
    try:
        prof_list = np.append(prof_list,i[0])
    except:
        prof_list = np.append(prof_list,0)
edu_colpd = pd.Series(prof_list)
test['Professional and business services'] = edu_colpd

trade_col = [i[0] for i in test_dic["Trade, transportation, and utilities"]]
trade_colpd = pd.Series(serv_col)
test['Trade, transportation, and utilities'] = trade_colpd

In [ ]:
test

In [ ]:
x = test[list(test.columns.values[:4])]  # Use only the first 3 columns to regress
y = test['DUI']  # Set y variable

model = sm.OLS(y, x).fit()
predictions = model.predict(x) # make the predictions by the model

print(model.summary())

In [ ]:
model23 = smf.ols(formula = 'y ~ x', data= test)
f_lin_reg = model23.fit()
f_lin_reg.summary()

In [ ]:
print(samp_db.keys())
# Pull desired crime and industry data
base00 = {}
for n in industry:  # Initializing keys
    base00[n] = ()
# while count > 5: # 5 for number of years available
for i in samp_db:
    for n in base00:
        for z in range(len(samp_db[i]['Industry'])):
            base00[n] = samp_db[i]['Industry'][z].loc[samp_db[i]['Industry'][z]['Industry'] == n]
print(base00.keys())
base00['Service-providing']